In [1]:
import numpy as np
import pandas as pd
import random

PLAYER = 1    # AI player
OPPONENT = -1

def check_winner(board):
    wins = [(0,1,2),(3,4,5),(6,7,8),
            (0,3,6),(1,4,7),(2,5,8),
            (0,4,8),(2,4,6)]
    for a,b,c in wins:
        if board[a] != 0 and board[a] == board[b] == board[c]:
            return board[a]
    return 0

def is_full(board):
    return 0 not in board

def minimax(board, depth, is_max):
    winner = check_winner(board)
    if winner == PLAYER:
        return 10 - depth
    if winner == OPPONENT:
        return depth - 10
    if is_full(board):
        return 0

    if is_max:
        best = -np.inf
        for i in range(9):
            if board[i] == 0:
                board[i] = PLAYER
                best = max(best, minimax(board, depth + 1, False))
                board[i] = 0
        return best
    else:
        best = np.inf
        for i in range(9):
            if board[i] == 0:
                board[i] = OPPONENT
                best = min(best, minimax(board, depth + 1, True))
                board[i] = 0
        return best

def find_best_move(board):
    best_val = -np.inf
    best_move = -1
    for i in range(9):
        if board[i] == 0:
            board[i] = PLAYER
            move_val = minimax(board, 0, False)
            board[i] = 0
            if move_val > best_val:
                best_val = move_val
                best_move = i
    return best_move

def generate_dataset(n_games=5000):
    X, y = [], []
    for _ in range(n_games):
        board = [0] * 9
        moves_played = random.randint(0, 4)  # random starting point
        for _ in range(moves_played):
            empty = [i for i, v in enumerate(board) if v == 0]
            if not empty:
                break
            move = random.choice(empty)
            board[move] = PLAYER if len(empty) % 2 == 1 else OPPONENT

        if check_winner(board) == 0 and not is_full(board):
            best_move = find_best_move(board)
            X.append(board.copy())
            y.append(best_move)
    return np.array(X), np.array(y)

X, y = generate_dataset(10000)

# Save to CSV
df = pd.DataFrame(X, columns=[f'pos{i}' for i in range(9)])
df['best_move'] = y
df.to_csv("tic_tac_toe_dataset.csv", index=False)
print("Dataset saved: tic_tac_toe_dataset.csv")


Dataset saved: tic_tac_toe_dataset.csv


In [2]:
dff=pd.read_csv('tic_tac_toe_dataset.csv')
dff.shape

(10000, 10)

In [7]:
X, y = generate_data(20)


np.save("X_tictactoe.npy", X)
np.save("y_tictactoe.npy", y)

In [ ]:
.shape

(10000,)

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
X_train

array([[-1,  1,  0, ...,  0, -1,  0],
       [-1,  1,  0, ...,  1,  0,  0],
       [-1, -1,  0, ...,  0,  0,  0],
       ...,
       [-1,  0,  0, ...,  0,  0,  0],
       [ 1,  0,  0, ...,  0,  0,  1],
       [ 0,  0,  0, ...,  0,  0,  0]])

In [37]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

In [38]:
X=np.load("X_tictactoe.npy")
y=np.load("y_tictactoe.npy")
y_cat = to_categorical(y, num_classes=9)

In [46]:
tf.keras.backend.clear_session()

In [47]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(9,)),
    Dense(64, activation='relu'),
    Dense(9, activation='softmax')  # 9 possible moves
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [48]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 9)              │           585 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,385 (21.04 KB)

 Trainable params: 5,385 (21.04 KB)

 Non-trainable params: 0 (0.00 B)

In [49]:
model.fit(X, y_cat, epochs=100, batch_size=32)


Epoch 1/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1676 - loss: 2.1890  
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2795 - loss: 2.0862 
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3368 - loss: 2.0157 
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3867 - loss: 1.9388 
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4156 - loss: 1.8821 
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4842 - loss: 1.8109 
Epoch 7/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4891 - loss: 1.7841 
Epoch 8/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5220 - loss: 1.7025 
Epoch 9/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5024 - loss: 1.6766 
Epoch 10/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5100 - loss: 1.5850 
Epoch 11/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5482 - loss: 1.5220 
Epoch 12/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

In [54]:
test_board = np.array([[-1, 1, 0, 0, 1, -1, 0, 0, 0]])
pred = model.predict(test_board)
print(pred)
best_move = np.argmax(pred)
print("Best move index:", best_move)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
[[1.2009899e-02 3.7036583e-02 1.0728574e-02 9.1780148e-02 2.4699615e-02
  5.3158574e-02 7.8315206e-02 6.9227052e-01 8.2289387e-07]]
Best move index: 7


In [52]:
model.save("tictactoe_model.h5")